## Backdoor Detection in DNN using STRIP Technique.

Group Members:
*Rahul Petkar* : **rsp431**
*Shubham Ingale* : **ski227**
*Cherian Thomas* : **kct298**
*Akhand Singh* : **aps646**

Note: Follow the same process for all badnets from B1, B2....Bn as defined in the Readme File (i.e; Substituting Required Paths before running the Script)

In [5]:
import keras
import sys
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
import random
import statistics
import math

In [6]:
# mount required drive for testing
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

def data_preprocess(x_data):
    return x_data/255

In [9]:
def PrintAccOnGivenModelAndData(model_filename, x_test, y_test):
  bd_model = keras.models.load_model(model_filename)
  clean_label_p = np.argmax(bd_model.predict(x_test), axis=1)
  class_accu = np.mean(np.equal(clean_label_p, y_test))*100
  print('Classification accuracy:', class_accu)

Path Substitution can be done in the below Code for Specific Badnets B1, B2....Bn



In [10]:
# This is where you paste to dataset link to generate the filename
ValidationData_filename = str("/content/drive/MyDrive/Colab Notebooks/clean_validation_data.h5")
model_filename = str("/content/drive/MyDrive/Colab Notebooks/sunglasses_bd_net.h5")
# These are given test datasets 
Poisoned_data_filename = str("/content/drive/MyDrive/Colab Notebooks/sunglasses_poisoned_data.h5")
clean_test_filename=str("/content/drive/MyDrive/Colab Notebooks/clean_test_data.h5")

In [11]:
# This function is used to calculate the detection boundary based on the Validation data
# The validation data is used to calculate Detection boundary
# It is achieved by superimposition of random validation images

def FindDetectionBoundary(model_filename, ValidationData_filename):
  x_validation, y_validation = data_loader(ValidationData_filename)
  x_validation = data_preprocess(x_validation)
  bd_model = keras.models.load_model(model_filename)
  N = 10
  lengthValidData = len(x_validation)  
  SuperImposeImg_Pred = np.zeros((lengthValidData, N))
  SuperImposeImg = []
  #SuperImposeImg = np.zeros(N)
  for idx,i in enumerate(x_validation):
    for _ in range(N):
      SuperImposeImg.append(cv2.addWeighted(i, 1, random.choice(x_validation), 1, 0,dtype = cv2.CV_32F))
    arr = np.array(SuperImposeImg)
    #print(SuperImposeImg)
    clean_label_p = np.argmax(bd_model.predict(arr), axis=1)
    SuperImposeImg.clear()
    #print(clean_label_p)
    newarr = list(clean_label_p)
    #for j in range(N):
    SuperImposeImg_Pred[idx] = clean_label_p

  Yi_Denom = y_validation.max() 
  Label_Map = {}
  Total_validation_Images = len(SuperImposeImg_Pred)
  Hsum = [] * Total_validation_Images
  Normal_H = [] * Total_validation_Images
  for i in range(Total_validation_Images):
    yi = []*N
    for j in SuperImposeImg_Pred[i]:
      if j not in Label_Map:
        Label_Map[j] = 0
      Label_Map[j] += 1
    idx = 0
    Hn = 0.0
    for Key,Val in Label_Map.items():
      yi_temp = float(Val/Yi_Denom) 
      yi.append(-1 * yi_temp * math.log2(yi_temp))
      Hn += yi[idx]
      idx+=1
    Label_Map.clear()
    Hsum.append(float(Hn))
    Normal_H.append(float(Hsum[i]/N))

  # Mean and Standard Deviation are calculated
  Mean_H = statistics.mean(Normal_H) 
  StdD_H = statistics.stdev(Normal_H)

  # The Z_score value for One percentile is considered to get Detection Boundary
  Z_score = -2.326
  DetectionBoundary = float(Mean_H + Z_score* StdD_H)
  return DetectionBoundary


In [12]:
# This the Predict class function
# It takes in input image and processes it
# The function then makes prediction on 10 images created by superimposing randomly selected 10 images from the Validation Data (Which is pre_processed i.e; divided by 255)
# The prediction entrophy is calculated and compared to Detection Boundary to generate Ouput Labels
def PredictClass(Test_Image):
  Test_Image = data_preprocess(Test_Image)
  N = 10
  SuperImposeImg = []  
  for j in range(N):
    SuperImposeImg.append(cv2.addWeighted(Test_Image, 1, random.choice(x_validation), 1, 0,dtype = cv2.CV_32F))
  arr = np.array(SuperImposeImg)
  TestDataPred = np.argmax(bd_model.predict(arr), axis=1)
  
  Yi_Denom = y_validation.max() 
  Label_Map = {}
  Total_Poison_Images = len(TestDataPred)
  yi = []*N

  # Backdoored Class N + 1 is calculated as below
  Backdoor = y_validation.max() + 1
  Output_Labels = 0
  Output_Labels = np.zeros(1)
  Hsum = 0.0
  for j in TestDataPred:
    if j not in Label_Map:
      Label_Map[j] = 0
    Label_Map[j] += 1
  idx = 0
  Hn = 0.0
  for Key,Val in Label_Map.items():
    yi_temp = float(Val/Yi_Denom) 
    yi.append(-1 * yi_temp * math.log2(yi_temp))
    Hn += yi[idx]
    idx+=1
  Hsum = float(Hn)
  CurrentEntrophy = float(Hsum/N)
  if CurrentEntrophy<=DetectionBoundary:
    Output_Labels[0] = Backdoor
  else:
    Output_Labels = np.argmax(bd_model.predict(np.array([Test_Image,] )), axis=1)
  return Output_Labels[0]

Run Below Scipt Once (Detection Boundary is calculated below)

In [13]:
# Calculate Detection Boundary
# This below code Should be run before using the PredictClass Function
# The Below code calculated Detection Boundary which is referenced by the PredictClass function
DetectionBoundary = FindDetectionBoundary(model_filename, ValidationData_filename)
x_validation, y_validation = data_loader(ValidationData_filename)
x_validation = data_preprocess(x_validation)
bd_model = keras.models.load_model(model_filename)
X_test, Y_test = data_loader(Poisoned_data_filename)

Class prediction of a Single Vanilla test Image from Poisoned Test Dataset

In [14]:
# Single Test Image 
X_test, Y_test = data_loader(Poisoned_data_filename)
Single_Test_Image = X_test[0,:,:,:]
print("Predicted Class: ", PredictClass(Single_Test_Image))

Predicted Class:  1283.0


The PredictClass function used below is to find out from the given **Poisoned Test Data Set**, as to how many images are categorised as backdoored.

In [15]:
def CalculateBackdoorAccuracy():
  Total_Backdoored_Images = 0
  # The Backdoored class representation according to project definition
  Backdoored_Class = y_validation.max() + 1
  for i in X_test:
    # PredictClass function takes one image as input and the return the output label on that image
    # PredictClass can be used independently to find out the Label on an individual input
    # PredictClass internally processes image by dividing it by 255, so make sure that a vanilla image is passed to it
    out = PredictClass(i)
    if out == Backdoored_Class:
      Total_Backdoored_Images+=1

  print("Total Backdoored Images Found: ", Total_Backdoored_Images)
  print("Total Images In the Test Set: ", len(X_test))
  print("Accuracy on Poisoned DataSet: ", float(Total_Backdoored_Images/len(X_test)) * 100)

In [16]:
# Calculate Backdoor accuracy
CalculateBackdoorAccuracy()

Total Backdoored Images Found:  12298
Total Images In the Test Set:  12830
Accuracy on Poisoned DataSet:  95.85346843335931


The PredictClass function used below to check the performance on **Clean Test Data** 

In [17]:
# Load new test data
X_test, Y_test = data_loader(clean_test_filename)
Total_Clean_Images = 0
# The Backdoored class representation according to project definition
Backdoored_Class = y_validation.max() + 1
for i in X_test:
  # PredictClass function takes one image as input and the return the output label on that image
  # PredictClass can be used independently to find out the Label on an individual input
  # PredictClass internally processes image by dividing it by 255, so make sure that a vanilla image is passed to it
  out = PredictClass(i)
  if out != Backdoored_Class:
    Total_Clean_Images+=1

print("Total Clean Images Found: ", Total_Clean_Images)
print("Total Images In the Clean Test Set: ", len(X_test))
print("Accuracy on Clean DataSet: ", float(Total_Clean_Images/len(X_test)) * 100)

Total Clean Images Found:  12302
Total Images In the Clean Test Set:  12830
Accuracy on Clean DataSet:  95.8846453624318


## Anonymous Badnet 1 

In [18]:
model_filename = str("/content/drive/MyDrive/Colab Notebooks/anonymous_1_bd_net.h5")
Poisoned_data_filename = str("/content/drive/MyDrive/Colab Notebooks/anonymous_1_poisoned_data.h5")
bd_model = keras.models.load_model(model_filename)

In [19]:
# Load new test data
X_test, Y_test = data_loader(Poisoned_data_filename)
# Load new test data
CalculateBackdoorAccuracy()

Total Backdoored Images Found:  8277
Total Images In the Test Set:  10264
Accuracy on Poisoned DataSet:  80.641075604053


### Multi Trigger Multi target Badnet

In [21]:
model_filename = str("/content/drive/MyDrive/Colab Notebooks/multi_trigger_multi_target_bd_net.h5")
Poisoned_data_filename1 = str("/content/drive/MyDrive/Colab Notebooks/Multi-trigger Multi-target/eyebrows_poisoned_data.h5")
Poisoned_data_filename2 = str("/content/drive/MyDrive/Colab Notebooks/Multi-trigger Multi-target/lipstick_poisoned_data.h5")
Poisoned_data_filename3 = str("/content/drive/MyDrive/Colab Notebooks/Multi-trigger Multi-target/sunglasses_poisoned_data.h5")
# global Variable - is referenced in other functions and should hold the required model before running PredictClass
bd_model = keras.models.load_model(model_filename)


In [22]:
# Load new test data
X_test, Y_test = data_loader(Poisoned_data_filename1)
# Calculate Backdoor accuracy
CalculateBackdoorAccuracy()

Total Backdoored Images Found:  3340
Total Images In the Test Set:  10264
Accuracy on Poisoned DataSet:  32.54091971940764


In [23]:
# Load new test data
X_test, Y_test = data_loader(Poisoned_data_filename2)
# Calculate Backdoor accuracy
CalculateBackdoorAccuracy()

Total Backdoored Images Found:  8082
Total Images In the Test Set:  10264
Accuracy on Poisoned DataSet:  78.74123148869836


In [24]:
# Load new test data
X_test, Y_test = data_loader(Poisoned_data_filename3)
# Calculate Backdoor accuracy
CalculateBackdoorAccuracy()

Total Backdoored Images Found:  9466
Total Images In the Test Set:  10264
Accuracy on Poisoned DataSet:  92.22525331254872


# **Results and Conclusion**

We observed that among all the backdoors provided the STRIP repaired network did poorly on only the eyebrow backdoor (32% accuracy). While on the rest the STRIP repaired network did an acceptably good job of backdoor detection (78-92% accuracy). It even fared well on the anonymous backdoor.